#### In this notebook we develop a pipeline for hyperparameter tuning for UMAP + HDBSCAN.

We need to tune the following params:

UMAP:
- n_neighbors: [2, 0.25*len(df)]
- min_dist: [0, 0.99]
- n_components: [2, n_features]
- metric: [9 metrics for binary data]

HDBSCAN:
- min_cluster_size:
- min_samples: 
Note: If you wish to explore different min_cluster_size settings with a fixed min_samples value, especially for larger dataset sizes, you can cache the hard computation, and recompute only the relatively cheap flat cluster extraction using the memory parameter, which makes use of joblib
- cluster_selection_epsilon: ?
[- alpha]X
[- leaf clustering, not EOM]


In [24]:
# c=20*20*161*9*20*20*20
# c=10*10*20*9*10*10*10
c=5*5*5*5*5*5

In [25]:
c

15625

In [26]:
c*10/(60*24*10)

10.850694444444445

In [8]:
import itertools

hyper_params = {
    'penalty': ['l1', 'l2'],
    'class_weight': [None, 'balanced'],
    'max_iter': [500, 1000, 30]
}


a = hyper_params.values()
combinations = list(itertools.product(*a))

In [9]:
len(combinations)

12

In [10]:
2*2*3

12

In [11]:
combinations

[('l1', None, 500),
 ('l1', None, 1000),
 ('l1', None, 30),
 ('l1', 'balanced', 500),
 ('l1', 'balanced', 1000),
 ('l1', 'balanced', 30),
 ('l2', None, 500),
 ('l2', None, 1000),
 ('l2', None, 30),
 ('l2', 'balanced', 500),
 ('l2', 'balanced', 1000),
 ('l2', 'balanced', 30)]